In [26]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os
_path = '.'
sys.path.append(os.path.join(_path))



def parse_tf_events_file(events_path, tag, time_horizon=None):
    ea = EventAccumulator(events_path).Reload()
    tag_values, steps = [], []
    for event in ea.Scalars(tag):
        if time_horizon is None or event.step <= time_horizon:
            tag_values.append(event.value)
            steps.append(event.step)
    return steps, tag_values

class ExperimentConfig:
    def __init__(self, base_path, experiment, seeds, algorithms, events, lr_list,
                 alphas, n_clients_list, availabilities, n_rounds, participations):
        """
        base_path: path to the folder logs/
        experiment: name of exp (str: mnist_CI_based_availability)
        seeds: list of seeds
        algorithms: list of algorithms
        events: list of events (e.g. ["global"])
        # b_values:
        lr_list: list of learning rates
        alphas: list of alpha values (for the iid - non iid -ness)
        n_clients_list: list of number of clients
        availabilities: list of availability types (e.g. ["local_mean", "global_mean", "random", "local_mean_green", "cyclic"])
        n_rounds: number of FL rounds
        participations: list with "unknown" and/or "known" (whether we work with the participation probabilities estimator or the true values)
        """
        self.base_path = base_path
        self.experiment = experiment
        self.seeds = seeds
        self.algorithms = algorithms
        self.events = events
        # self.b_values = b_values
        self.lr_list = lr_list
        self.alphas = alphas
        self.n_clients_list = n_clients_list
        self.availabilities = availabilities
        self.n_rounds = n_rounds
        self.participations = participations

    def get_event_dir(self, algo, lr, seed, event, alpha, n_clients, availability, n_rounds, participation):
        """
        algorithm: algorithm (str:fedavg, defvarp or fedstale)
        lr: learning rate (str)
        b: beta (str)
        seed:
        event:
        """
        path = f"{self.base_path}/{self.experiment}/clients_{n_clients}/{availability}"
        # path += f"/{algo}/b_{b}" if algo == "mixture" else f"/{algo}" # in case we vary beta
        path += f"/{algo}/{participation}_participation_probs"
        path += f"/alpha_{alpha}/lr_{lr}/seed_{seed}/rounds_{n_rounds}/train/{event}"

        path = os.path.join(self.base_path, self.experiment, "clients_"+n_clients, availability,
                     algo, participation+"_participation_probs", "alpha_"+alpha, "lr_"+lr, 
                     "seed_"+seed, "rounds_"+n_rounds, "train", event)

        return path

# Load and Process Experiment Results
def load_experiment_results(config):
    results = [] 
    for lr in tqdm(config.lr_list, desc="Processing experiments"):
        # time_horizon = time_horizons[p]  
        for algorithm in config.algorithms:
            # b_loop = config.b_values if algorithm == 'mixture' else [None] # in case we vary beta
            # for b in b_loop:
            for event in config.events:
                for seed in config.seeds:
                    for a in config.alphas:
                        for n_c in config.n_clients_list:
                            for av in config.availabilities:
                                for part in config.participations:

                                    event_dir = config.get_event_dir(algorithm, lr, seed, 
                                                                        event, a, n_c, av, 
                                                                        config.n_rounds, part)                                    
                                    # files = os.listdir(event_dir)

                                    if os.path.exists(event_dir):
                                        # _, values = parse_tf_events_file(event_dir, tag="Test/Metric", time_horizon=time_horizon)
                                        _, test_accuracy_values = parse_tf_events_file(event_dir, tag="Test/Metric")
                                        # tag can be: 'Train/Loss', 'Train/Metric', 'Test/Loss', 'Test/Metric'
                                        max_accuracy = np.array(test_accuracy_values).max() * 100
                                        results.append({
                                            "algorithm": algorithm, "availability": av,
                                            "alpha": a, "participation": part,
                                            "max_test_accuracy": float(max_accuracy),
                                            "test_accuracy": test_accuracy_values,
                                            "seed": int(seed),
                                            "lr": lr, "event": event, "n_clients": n_c
                                        })

                                        # "b": float(b) if b else np.nan # in case we vary beta
    return pd.DataFrame(results)

# Main
config = ExperimentConfig(base_path=os.path.join('.', 'logs'), experiment="mnist_CI_based_availability", seeds=["12"],
                          algorithms=["fedavg", "fedvarp", "fedstale"], events=["global"],
                          lr_list=["5e-3"], alphas=["100000", "0.1"], n_clients_list=["7"],
                          availabilities=["local_mean", "global_mean", "random", "local_mean_green", "cyclic"],
                          n_rounds="100", participations=["known", "unknown"]
                         )


results_df = load_experiment_results(config)
results_df
#  b_values=["0.2", "0.5", "0.8"]

# ./logs/mnist_CI_based_availability/clients_7/local_mean/fedavg/known_participation_probs/alpha_100000/lr_5e3/seed_12/rounds_100/train/global
# \logs\mnist_CI_based_availability\clients_7\local_mean\fedavg\known_participation_probs\alpha_100000\lr_5e-3\seed_12\rounds_100\train\global

Processing experiments: 100%|██████████| 1/1 [00:00<00:00,  2.42it/s]


,algorithm,availability,alpha,participation,max_test_accuracy,test_accuracy,seed,lr,event,n_clients
0,fedavg,local_mean,100000,known,93.121415,"[0.08441896736621857, 0.25169357657432556, 0.3...",12,5e-3,global,7
1,fedavg,local_mean,100000,unknown,92.131317,"[0.08441896736621857, 0.21782177686691284, 0.2...",12,5e-3,global,7
2,fedavg,global_mean,100000,known,93.121415,"[0.08441896736621857, 0.25169357657432556, 0.3...",12,5e-3,global,7
3,fedavg,global_mean,100000,unknown,92.131317,"[0.08441896736621857, 0.21782177686691284, 0.2...",12,5e-3,global,7
4,fedavg,random,100000,known,93.121415,"[0.08441896736621857, 0.25169357657432556, 0.3...",12,5e-3,global,7
5,fedavg,random,100000,unknown,92.131317,"[0.08441896736621857, 0.21782177686691284, 0.2...",12,5e-3,global,7
6,fedavg,local_mean_green,100000,known,93.121415,"[0.08441896736621857, 0.25169357657432556, 0.3...",12,5e-3,global,7
7,fedavg,local_mean_green,100000,unknown,92.131317,"[0.08441896736621857, 0.21782177686691284, 0.2...",12,5e-3,global,7
8,fedavg,cyclic,100000,known,93.121415,"[0.08441896736621857, 0.25169357657432556, 0.3...",12,5e-3,global,7
9,fedavg,cyclic,100000,unknown,92.131317,"[0.08441896736621857, 0.21782177686691284, 0.2...",12,5e-3,global,7


In [ ]:

# from tensorflow.python.summary.summary_iterator import summary_iterator

# for summary in summary_iterator("./logs/mnist_CI_based_availability/clients_7/local_mean/fedavg/known_participation_probs/alpha_0.1/lr_5e-3/seed_12/rounds_100"):
#     print(summary)

folder = './logs/mnist_CI_based_availability/clients_7/local_mean/fedavg/known_participation_probs/alpha_0.1/lr_5e-3/seed_12/rounds_100/train/global/events.out.tfevents.1717493789.C11-8C4G9B45HOX.12548.7'
# folder = './logs'

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
event_acc = EventAccumulator(folder)
event_acc.Reload()
# Show all tags in the log file
print(event_acc.Tags())

# w_times, step_nums, vals = zip(*event_acc.Scalars('Train/Loss'))

# print(*event_acc.Scalars('Train/Loss'))


event_acc.Scalars('Train/Loss')

In [2]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

def parse_tf_events_file(events_path, tag, time_horizon=None):
    # extract the list of steps (FL rounds) and the list or corresponding values
    ea = EventAccumulator(events_path).Reload()
    tag_values, steps = [], []
    for event in ea.Scalars(tag):
        if time_horizon is None or event.step <= time_horizon:
            tag_values.append(event.value)
            steps.append(event.step)
    return steps, tag_values


class ExperimentConfig:
    def __init__(self, base_path, experiment, seeds, algorithms, events, heterogeneities, b_values, lr_dict
                 ):
        self.base_path = base_path
        self.experiment = experiment
        self.seeds = seeds
        self.algorithms = algorithms
        self.events = events
        self.heterogeneities = heterogeneities
        self.b_values = b_values
        self.lr_dict = lr_dict

    def get_event_dir(self, p, h, algorithm, lr, b, seed, event):
        path = f"{self.base_path}/{self.experiment}/p_{p}/h_{h}"
        path += f"/{algorithm}/b_{b}" if algorithm == "mixture" else f"/{algorithm}"
        path += f"/lr_{lr}/seed_{seed}/train/{event}"
        return path

parse_tf_events_file(folder, 'Train/Loss')

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100],
 [2.3053338527679443,
  2.260401725769043,
  2.2076103687286377,
  2.0828468799591064,
  1.8212461471557617,
  1.6342271566390991,
  1.4341044425964355,
  1.2726633548736572,
  1.1403427124023438,
  1.178110957145691,
  0.8990321159362793,
  0.8210407495498657,
  0.7579436898231506,
  0.7097863554954529,
  0.6728664040565491,
  0.6978819370269775,
  0.6431033611297607,
  0.6291451454162598,
  0.7222